In [1]:
import gym
import time
import pygame
from sklearn.preprocessing import normalize
import tensorflow as tf
import tflearn
import numpy as np
import random
import matplotlib
import seaborn as sns
%matplotlib inline

In [2]:
env = gym.make("MsPacman-v0")

In [3]:
"Num actions: " + str(env.action_space.n)

'Num actions: 9'

In [4]:
"Observation space: " + str(env.observation_space)

'Observation space: Box(210, 160, 3)'

In [5]:
observation = tflearn.input_data(shape=[None, 210, 160, 3])
net = tflearn.max_pool_2d(observation, 2)
w1 = tflearn.conv_2d(observation, 32, 3, padding='valid', activation='relu')
net = tflearn.max_pool_2d(w1, 2)
net = tflearn.conv_2d(net, 32, 3, padding='valid', activation='relu')
net = tflearn.max_pool_2d(net, 2)
net = tflearn.conv_2d(net, 64, 3, padding='valid', activation='relu')
net = tflearn.max_pool_2d(net, 2)
net = tflearn.conv_2d(net, 64, 3, padding='valid', activation='relu')
net = tflearn.max_pool_2d(net, 2)
net = tflearn.conv_2d(net, 128, 3, padding='valid', activation='relu')
net = tflearn.max_pool_2d(net, 2)
net = tflearn.conv_2d(net, 128, 3, padding='valid', activation='relu')
net = tflearn.fully_connected(net, 1028, activation='relu')
out = tflearn.fully_connected(net, env.action_space.n, activation="softmax")

reward_holder = tf.placeholder(tf.float32, [None])
action_holder = tf.placeholder(tf.int32, [None])

responsible_outputs = tf.gather(tf.reshape(out, [-1]), tf.range(0, tf.shape(out)[0] * tf.shape(out)[1], env.action_space.n) + action_holder)

loss = -tf.reduce_mean(tf.log(responsible_outputs + 0.0001) * reward_holder)

optimizer = tf.train.AdamOptimizer(learning_rate=0.0005)
update = optimizer.minimize(loss)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


/home/ejmejm/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [6]:
gamma = 0.99

def discount_reward(rewards):
    running_reward = 0
    result = np.zeros_like(rewards)
    for i in reversed(range(len(rewards))):
        result[i] = rewards[i] + gamma * running_reward
        running_reward += rewards[i]
    return result

In [7]:
def pixelNorm(img):
    return img / 255.

In [8]:
num_episodes = 2000
decay = 10
max_time = 1200
all_rewards = []
saver = tf.train.Saver()
train_data = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_episodes):
        obs = env.reset()
        obs = pixelNorm(obs)
        episode_reward = 0
        ep_history = []
        for j in range(max_time+1):
            #Choose an action
            a_one_hot = sess.run(out, feed_dict={observation: [obs]}).reshape(env.action_space.n)
            #print(a_one_hot)
#             action = np.random.choice(a_one_hot, p=a_one_hot)
#             action = np.argmax(a_one_hot == action)
            if random.random() < (decay/(decay+i)):
                action = np.random.choice(9)
            else:
                action = action = np.argmax(a_one_hot)
            #print(action)
            #env.render()
            obs1, r, d, _ = env.step(action)
            obs1 = pixelNorm(obs)
            ep_history.append([obs, r, action])
            obs = obs1
            episode_reward += r
            if d == True or j == max_time:
                all_rewards.append(episode_reward)
                ep_history = np.array(ep_history)
                ep_history[:, 1] = discount_reward(ep_history[:, 1])
                train_data.extend(ep_history)
                if i % 1 == 0 and i != 0:
                    train_data = np.array(train_data)
                    
                    shf = np.random.permutation(train_data[:, 1].shape[0])
                    train_data[:, 0] = train_data[:, 0][shf]
                    train_data[:, 1] = train_data[:, 1][shf]
                    train_data[:, 2] = train_data[:, 2][shf]
                    
                    for prt in range(0, len(train_data), 16):
                        _, q, q2 = sess.run([update, out, responsible_outputs], feed_dict={observation: np.vstack(train_data[prt:prt+16, 0]).reshape(-1, 210, 160, 3),
                                                        reward_holder: train_data[prt:prt+16, 1],
                                                        action_holder: train_data[prt:prt+16, 2]})
                    train_data = []
                break
                
        if i % 20 == 0 and i != 0:
            print("Reward: " + str(np.mean(all_rewards[-20:])))
            
    saver.save(sess, "/tmp/model.ckpt")

Reward: 296.5
Reward: 382.5
Reward: 326.0
Reward: 270.0
Reward: 317.5
Reward: 281.5
Reward: 297.0
Reward: 189.5
Reward: 210.5
Reward: 212.0
Reward: 223.0
Reward: 186.0
Reward: 239.5
Reward: 183.5
Reward: 205.5
Reward: 173.5
Reward: 139.5
Reward: 201.5
Reward: 121.0
Reward: 168.0
Reward: 185.5
Reward: 149.5
Reward: 136.0
Reward: 113.0
Reward: 155.5
Reward: 143.0
Reward: 226.5
Reward: 121.5
Reward: 127.5
Reward: 139.0
Reward: 155.0
Reward: 120.5
Reward: 126.0
Reward: 98.5
Reward: 126.0
Reward: 123.0
Reward: 135.0
Reward: 119.5
Reward: 130.5
Reward: 104.5
Reward: 111.5
Reward: 110.5
Reward: 107.5
Reward: 94.0
Reward: 111.0
Reward: 113.0
Reward: 106.0
Reward: 99.0
Reward: 119.5
Reward: 97.5
Reward: 109.5
Reward: 82.5
Reward: 91.5
Reward: 121.0
Reward: 95.0
Reward: 113.5
Reward: 112.5
Reward: 89.5
Reward: 103.5
Reward: 84.0
Reward: 123.5
Reward: 102.0
Reward: 107.0
Reward: 73.0
Reward: 80.5
Reward: 74.5
Reward: 86.5
Reward: 93.5
Reward: 88.5
Reward: 84.5
Reward: 88.0
Reward: 104.0
Reward: 7

In [9]:
avg_reward = [np.mean(all_rewards[i-10:i+10]) for i in range(10, len(all_rewards))]
sns.plt.plot(avg_reward[::10])

AttributeError: module 'seaborn' has no attribute 'plt'

In [ ]:
max_time = 200
saver = tf.train.Saver()
    
with tf.Session() as sess:
    saver.restore(sess, "/tmp/model.ckpt")
    #Show the results
    for i in range(10):
        obs = env.reset()
        episode_reward = 0
        for j in range(max_time):
            #Choose an action
            a_one_hot = sess.run(out, feed_dict={observation: [obs]}).reshape(9)
            action = np.random.choice(a_one_hot, p=a_one_hot)
            action = np.argmax(a_one_hot == action)
            env.render()
            time.sleep(0.005)
            obs, r, d, _ = env.step(action)
            episode_reward += r
            if d == True:
                break
        print(episode_reward)

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
60.0
60.0
60.0
60.0
